<a href="https://colab.research.google.com/github/fekonrad/CombOptLayer/blob/main/Demo/COptLayer_Warcraft_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a short demo on how to use the COptLayer for the toy problem of finding shortest paths on Warcraft maps. To illustrate what the maps, graphs and paths look like, see the plots below:

In [ ]:
# TODO: Implement Warcraft Dataset

In [ ]:
# TODO: Load and plot a sample map

# Using the COptLayer
First, let's install the COptLayer repository:

In [ ]:
# TODO: Clone repo

We implement a very basic CNN to estimate the vertex costs of the map. Since this problem is not very complex, a small model will suffice.

In [ ]:
# TODO: Add implementation of CNN

Now we just have to implement our solver for finding shortest paths (given vertex weights) on 2D grids, where  the possible moves at each point are up, down, left, right and diagonal.

We have to make sure that the input of our solver is compatible with what our statistical model (the CNN) returns as outputs, i.e. in this case a `torch.tensor` of shape `(b, 1, h, w)`.

**Remark:**
The COptLayer will have to solve multiple instances of the combinatorial problem at once. Therefore it is clever to try to implement the solver using parallelizable operations, instead of trying to solve the problems sequentially. This means either sticking to what `torch` already has implemented or writing your own parallelized methods e.g. using CUDA.

In [ ]:
# TODO: Implement Solver

Now we have everything we need to build the model!

In [ ]:
# TODO: Build full model

The training routine now works like any other training routine in torch!

In [ ]:
# TODO: Implement training routine and monitor sample map.

# Results
...